# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [49]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [57]:
city_data = pd.read_csv("..\WeatherPy\Output\City_Data_Output.csv")
cleancity_data = city_data.dropna(inplace=False)
cleancity_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hobart,-42.88,147.33,64.99,60.80,72,75,1.12,AU,1609711992
1,1,qaanaaq,77.48,-69.36,2.01,2.01,83,82,3.53,GL,1609712078
2,2,lavrentiya,65.58,-171.00,-6.02,-6.02,90,85,15.14,RU,1609712079
3,3,tiksi,71.69,128.87,-23.73,-23.73,84,43,3.38,RU,1609712080
4,4,wattegama,6.80,81.48,75.20,75.20,100,75,5.82,LK,1609712082


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [97]:
center_lat = cleancity_data["Lat"].mean()
center_lng = cleancity_data["Lng"].mean()
center_coordinates = (center_lat,center_lng)

gmaps.configure(api_key=g_key)
locations = cleancity_data[["Lat","Lng"]].astype(float)
humidity = cleancity_data["Humidity"].astype(float)
max_humidity = cleancity_data["Humidity"].max()

figure_layout = {
    'width': '1000px',
    'height': '750px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(center=center_coordinates, layout=figure_layout, zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=max_humidity,point_radius=2)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='750px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [86]:
vacation_cities = (cleancity_data.loc[(cleancity_data["Max Temp"]<83) & (cleancity_data["Max Temp"]>68) & (cleancity_data["Min Temp"]>68) & (cleancity_data["Wind Speed"]<10) & (cleancity_data["Cloudiness"]<20)]).reset_index(drop=True)
#vacation_cities.head()
len(vacation_cities)

32

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [87]:
hotel_df = vacation_cities
hotel_df.reset_index()
hotel_df["Hotel"] = ""

for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    city = row["City"]
    search_lat = row["Lat"]
    search_lng = row["Lng"]
    search_coordinates = f"{search_lat},{search_lng}"
    params = {"location": search_coordinates, "radius": 5000, "type": "lodging", "key": g_key}
    response = requests.get(base_url,params=params).json()
    results = response['results']
    try:
        hotel_df.loc[index, 'Hotel'] = results[0]['name']
    except(KeyError, IndexError):
        print(f"No hotel in 5000m radius for {city}")
        print("---------------------------")
    
    
    
    
    


No hotel in 5000m radius for yuli
---------------------------
No hotel in 5000m radius for kidal
---------------------------
No hotel in 5000m radius for bara
---------------------------
No hotel in 5000m radius for manyana
---------------------------


In [88]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
0,21,port alfred,-33.59,26.89,73.00,73.00,93,18,3.83,ZA,1609712104,The Halyards Hotel
1,97,kolokani,13.57,-8.03,78.73,78.73,22,0,6.96,ML,1609712204,Lycee Famolo Coulibalibali De Kolokani
2,101,oussouye,12.48,-16.55,75.20,73.40,88,0,4.70,SN,1609712208,Auberge du Routard
3,103,hilo,19.73,-155.09,80.60,72.00,57,1,4.70,US,1609712030,Hilo Hawaiian Hotel
4,105,mahanoro,-19.90,48.80,72.18,72.18,81,13,4.00,MG,1609712213,BUNGALOW MAHAMANINA
5,117,camapua,-19.53,-54.04,81.70,81.70,58,12,4.34,BR,1609712229,Comercial Estrela
6,127,dhidhdhoo,6.88,73.10,81.23,81.23,70,5,7.07,MV,1609712241,Dhidhoo beach
7,169,yuli,9.70,10.27,79.00,79.00,20,0,4.61,NG,1609712293,
8,185,lazaro cardenas,17.96,-102.20,82.85,82.85,52,0,9.82,MX,1609712313,Collection O Hotel Quinta Antigua
9,196,diffa,13.32,12.61,77.16,77.16,17,0,6.67,NE,1609712327,ProDAF/Diffa


In [98]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [99]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0,150,0,0.4)',stroke_color='rgba(0,0,150,0.4)', scale=5,info_box_content = hotel_info)
hotel_center_lat = hotel_df["Lat"].mean()
hotel_center_lng = hotel_df["Lng"].mean()
hotel_center_coordinates = (hotel_center_lat,hotel_center_lng)

figure_layout = {
    'width': '900px',
    'height': '675px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

# Display figure
fig = gmaps.figure(center=hotel_center_coordinates, layout=figure_layout, zoom_level = 2)
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='675px', margin='0 auto 0 auto', padding='1px', wi…